## Task 0

In [1]:
%pip install duckdb pandas

Note: you may need to restart the kernel to use updated packages.


c:\Users\Pathan Imran Khan\Documents\github\SQL-Lab_Afreen_DE_25\.venv\Scripts\python.exe: No module named pip


In [6]:
# import liabraries
# sqlite3 helps Python open the sqlite file
# duckdb helps to create a duckdb database
# pandas use to load and move the data

import sqlite3
import duckdb
import pandas  as pd
import matplotlib.pyplot as pyplot

In [7]:
sqlite_path = "..sakila.sqlite"
duckdb_path = "..sakila.duckdb"

In [9]:
# Open sqlite database to read tables

# Connnect to sqlite database, bridge btwn Python & sakila.sqlite file
sqlite_conn = sqlite3.connect(sqlite_path)   

# Read table names: asks SQLite to show all table names inside this data
tables_query="SELECT name FROM sqlite_master WHERE type ='table';"

# pd.read_sql runs the sql query & makes it easy to view the table names.
tables = pd.read_sql(tables_query, sqlite_conn)

# Display the list of tables so we know what exists in sqlite database
tables


,name


### Connect SQLite

In [ ]:
sqlite_con = sqlite3.connect(sqlite_path)
sqlite_con

In [ ]:
# import duckdb

# with duckdb.connect("databases/sakila.duckdb") as conn:
# # conn.sql("""

# # CREATE TABLE IF NOT EXISTS bookings (
# #     title STRING,
# #     rating BOOLEAN,
# #     length FLOAT
# #     description STRING
# # );
# # """)

## Task  1

#### a) Movies longer than 180 min, show the title and its length

In [ ]:
# Query movies longer than 180 minutes
query = """ 
SELECT title, length
FROM film
WHERE length > 180
ORDER BY length DESC;
"""

# -- This SQL selects movies where the length column is above 180 minutes.